#### tf.Variable() vs tf.get_variable()
1. tf.Variable() 受tf.name_scope()作用；tf.get_variable()只受tf.name_space()作用
2. tf.Variable() name可以一样，tf内部有机制保证不会重名；tf.get_variable()

In [ ]:
### 保存变量

import tensorflow as tf

# Create some variables.
v1 = tf.get_variable("v1", shape=[3], initializer = tf.zeros_initializer)
v2 = tf.get_variable("v2", shape=[5], initializer = tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dec_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, initialize the variables, do some work, and save the
# variables to disk.
with tf.Session() as sess:
  sess.run(init_op)
  # Do some work with the model.
  sess.run(inc_v1)
  sess.run(dec_v2)
  # Save the variables to disk.
  save_path = saver.save(sess, "/data/tmp/model.ckpt")
  print("Model saved in file: %s" % save_path)


In [ ]:
### 加载变量
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

In [ ]:
# 加载部分变量
tf.reset_default_graph()

# Create some variables.
v1 = tf.get_variable("v1", shape=[3])
v2 = tf.get_variable("v2", shape=[5])

# Add ops to save and restore all the variables.
saver = tf.train.Saver([v2,])

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Check the values of the variables
  v1.initializer.run()
  print("v1 : %s" % v1.eval())
  print("v2 : %s" % v2.eval())

#### 概念
```
MetaGraph: dataflow graph, plus its associated variables, assets, and signatures
MetaGraphDef: protocol buffer representation of a MetaGraph
signature: the set of inputs to and outputs from a graph
```

In [ ]:
# build the graph for train & eval

tf.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
import datetime
mnist = input_data.read_data_sets("/data/mnist", one_hot=True)

# 全局参数
FC_NODE_N = 1024
BATCH_SIZE = 100
KEEP_PROB = 0.7
BATCH_N = mnist.train.num_examples // BATCH_SIZE
PADDING = 'SAME'

# w*x + b
def var_weight(shape):
  initial = tf.random_normal(shape=shape, stddev=0.1)
  return tf.Variable(initial)

def var_bias(shape):
  return tf.Variable(tf.constant(0.1, shape=shape))

# 定义二维卷积层
def conv2d(x, w):
  return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding=PADDING)

# 定义池化层
def pool_2x2(x):
  return tf.nn.max_pool(x, strides=[1, 2, 2, 1], ksize=[1, 2, 2, 1], padding=PADDING)

# 图像特征输入层
with tf.name_scope("input"):
  x = tf.placeholder(tf.float32, [None, 784])
  y = tf.placeholder(tf.float32, [None, 10])

  x_image = tf.reshape(x, shape=[-1, 28, 28, 1])

# layer1
with tf.name_scope("conv_1"):
  w_conv1 = var_weight([5, 5, 1, 32])
  b_conv1 = var_bias([32])

  # with tf.name_scope("max_pool_1"):
  h_conv1 = tf.nn.relu(conv2d(x_image, w_conv1) + b_conv1)
  h_pool1 = pool_2x2(h_conv1)

# layer2
with tf.name_scope("conv_2"):
  w_conv2 = var_weight([5, 5, 32, 64])
  b_conv2 = var_bias([64])

  # with tf.name_scope("max_pool_2"):
  h_conv2 = tf.nn.relu(conv2d(h_pool1, w_conv2) + b_conv2)
  h_pool2 = pool_2x2(h_conv2)

# full connected layer1
with tf.name_scope("full_connected_1"):
  w_fc1 = var_weight([7 * 7 * 64, FC_NODE_N])
  b_fc1 = var_bias([FC_NODE_N])

  h_pool2_flat = tf.reshape(h_pool2, [-1, 7 * 7 * 64])
  h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, w_fc1) + b_fc1)
  keep_prob = tf.placeholder(tf.float32)
  h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)

# full connected layer2
with tf.name_scope("full_connected_2"):
  w_fc2 = var_weight([1024, 10])
  b_fc2 = var_bias([10])

  output = tf.matmul(h_fc1_drop, w_fc2) + b_fc2

with tf.name_scope("softmax"):
  cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output))
  tf.summary.scalar("cross_entropy", cross_entropy)

with tf.name_scope("train"):
  learning_rate = tf.Variable(0.001, tf.float32)
  train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

  accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output, 1), tf.argmax(y, 1)), dtype=tf.float32))
  tf.summary.scalar("train_accuracy", accuracy)

merged = tf.summary.merge_all()


In [ ]:
# 训练 & 保存模型
MAX_STEP_NOIMPOROVEMENT = 100
MAX_ACC = 0
step_noimporvement = 0
MODEL_PATH = "/data/mnist_saver/mnist_cnn"

saver = tf.train.Saver()

with tf.Session() as sess:
  train_writer = tf.summary.FileWriter('logs/mnist/train', sess.graph)
  test_writer = tf.summary.FileWriter('logs/mnist/test', sess.graph)
  sess.run(tf.global_variables_initializer())
  for epoch in range(10000):
    sess.run(tf.assign(learning_rate, 0.001 * (0.98 ** int(epoch/10))))
    batch_x, batch_y = mnist.train.next_batch(BATCH_SIZE)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y, keep_prob: KEEP_PROB})

    summary = sess.run(merged, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
    train_writer.add_summary(summary, epoch)

    batch_x, batch_y = mnist.test.next_batch(BATCH_SIZE)
    summary = sess.run(merged, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
    test_writer.add_summary(summary, epoch)
    
    if epoch % 10 == 0:
      train_acc = sess.run(accuracy, feed_dict={x: mnist.train.images, y: mnist.train.labels, keep_prob: 1.0})
      test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
      print datetime.datetime.now(), "epoch [%d], lr [%f], train_acc[%f], test_acc[%f] %s" % \
            (epoch, sess.run(learning_rate), train_acc, test_acc, '*' if test_acc > MAX_ACC else '')
      if (test_acc > MAX_ACC):
        MAX_ACC = test_acc
        step_noimporvement = 0
        saver.save(sess, MODEL_PATH, global_step=epoch)
    
      if (step_noimporvement > MAX_STEP_NOIMPOROVEMENT):
        break
      else:
        step_noimporvement += 1

In [ ]:
# 加载模型

# 找到最后存储的模型
import os

def print_acc():
  train_acc = sess.run(accuracy, feed_dict={x: mnist.train.images, y: mnist.train.labels, keep_prob: 1.0})
  test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
  print datetime.datetime.now(), "train_acc[%f], test_acc[%f]" % (train_acc, test_acc)

# 重新初始化参数，查看准确率
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print_acc()
    
# 加载模型后，查看准确率
  ckpt = tf.train.get_checkpoint_state(os.path.join(os.path.split(MODEL_PATH)[0:-1])[0])
  if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print_acc()
  pass